# Schema of the technic

![](2023-12-30-23-52-11.png)

In [ ]:
# 1- api_key .env aldim
# 2- PDF sayfalarini birlestirip texts adinda topladi
# 3- CharacterTextSplitter ile chunklara böldü- OpenAI text-ada002 modelini kullandi orda tokensize 1000 idi  bizde chunk size 1000 aldik  gecemeyiz bu sayiyi
# 4- OpenAIEmbeddings() bunu cagirdik
# 5- docsearch = FAISS.from_texts(texts, embeddings) bunun ile PDF dosysini OpenAI tanimis oldu yani PDF embeddingslere ayrildi
# 6- chain = load_qa_chain(OpenAI(), chain_type='stuff') langchainin load_qa_chain sayesinde sorular sormaya baslayabildik

In [1]:
# eger fazla sayida PDF dosyalarin varsa bunlarin hepsini modeline train etmen zor olur.
# onlarin icinden gerekli yeri alip LLm onlardan anlamli bir cevap verecek
# önce PDF icerigi mesela 10 parcaya bölünür
# chunks sayisi token size dan kücük olmali (bu bilgi emin degilim)

# sonra her bir chunk'i embeddings vectörlerine ceviriyoruz. 
# galiba bunlari modelin tokenizeri ile yapiyoruz
# burda ChatGPT embeddingsi kullanacagiz
# bu sekilde bir knowladge base olusturacagi. galiba bu sekilde PDF lerin konusu anlasilmis olacak model tarafindan

# eger user soru sordugunda soruda OpenAI embeddings tarafindan embeddings vectörlerine cevrilecek
# cünkü ayni embeddings tarafinan yapilmasi gerekiyorki sayilarin anlamlari ayni olsun
# sonra semantic search bölümünde sorunun anlami bulunmaya calisiliyor
# ve knowledge base de bir sorgu ile bulunmaya calisiliyor

# ve PDF ler icinden bulunan sonuclarin bir rank degerleri oluyor
# bu degerler sonra LLM generative AI bölümüne aktariliyor


In [2]:
# ! pip install openai
# ! pip install PyPDF2
# pip install faiss-cpu

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

# ElasticVectorSearch : bu bir database denebilir. metinleri güclü ve hizli bir sekilde aramaniza yardimci olabilir
# Pinecone :  vectör arama icin tasarlanmistir
# Weaviate ; metin,kod, görüntü, ses gibi dosyalari depolayabilir.bir vectör database diyebiirz. 

# FAISS : Facebook tarafindan gelistirilmis vectör database lerde benzerlik aramasi yapar. 
# galiba PDF lerden ortak cevabi cikarmak icin olabilir



In [4]:
from dotenv import load_dotenv
import os

# .env dosyasını yükle
load_dotenv()

# Ortam değişkenini al
openai_api_key = os.getenv('OPENAI_API_KEY')


In [5]:
reader = PdfReader('/Users/veyselaytekin/Desktop/byte/NLP-LLM/ChatGPT_with_Langchain_for_own_PDF_files/2023_GPT4All_Technical_Report.pdf')
reader

In [6]:
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text +=text

In [7]:
raw_text[:300]

'GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nYuvanesh Anand\nyuvanesh@nomic.aiZach Nussbaum\nzanussbaum@gmail.com\nBrandon Duderstadt\nbrandon@nomic.aiBenjamin Schmidt\nben@nomic.aiAndriy Mulyar\nandriy@nomic.ai\nAbstract\nThis preliminary technical repo'

In [16]:
text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 1000, # bu openai tokensize boyutunu gecmemesi gerekir. her bir 1000 karakter 1 chunks oluyor
    chunk_overlap = 200, # bu chunks lar arasi ortak olan karakter sayisi asagida aciklamasi var
    length_function = len,
)

texts = text_splitter.split_text(raw_text)
texts[0]

'GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nYuvanesh Anand\nyuvanesh@nomic.aiZach Nussbaum\nzanussbaum@gmail.com\nBrandon Duderstadt\nbrandon@nomic.aiBenjamin Schmidt\nben@nomic.aiAndriy Mulyar\nandriy@nomic.ai\nAbstract\nThis preliminary technical report describes the\ndevelopment of GPT4All, a chatbot trained\nover a massive curated corpus of assistant in-\nteractions including word problems, story de-\nscriptions, multi-turn dialogue, and code. We\nopenly release the collected data, data cura-\ntion procedure, training code, and final model\nweights to promote open research and repro-\nducibility. Additionally, we release quantized\n4-bit versions of the model allowing virtually\nanyone to run the model on CPU.\n1 Data Collection and Curation\nWe collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-'

In [18]:
texts[1]
# 0. chunkta son cümlesi bunun ilk cümlesi olmus. normalde bunu yapmak zorunda degiliy ama
# videoda faydali oluyor dedi

'We collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-\nple of questions/prompts by leveraging three pub-\nlicly available datasets:\n• The unified chip2 subset of LAION OIG.\n• Coding questions with a random sub-sample\nof Stackoverflow Questions\n• Instruction-tuning with a sub-sample of Big-\nscience/P3\nWe chose to dedicate substantial attention to data\npreparation and curation based on commentary in\nthe Stanford Alpaca project (Taori et al., 2023).\nUpon collection of the initial dataset of prompt-\ngeneration pairs, we loaded data into Atlas for data\ncuration and cleaning. With Atlas, we removed all\nexamples where GPT-3.5-Turbo failed to respond\nto prompts and produced malformed output. This\nreduced our total number of examples to 806,199\nhigh-quality prompt-generation pairs. Next, we\ndecided to remove the entire Bigscience/P3 sub-'

In [17]:
len(texts) # toplamda 8 chunks var yani bu file yaklasik 8000 harf(karakter) civarinda

8

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()
# embeddings

# bu embeddings i yazdirinca model ile ilgili bilgiler ve chunks sayisi yaziyor. 1000 galiba bizde bundan dolayi 1000 i gecmedik
# bunu print yapmiyorum cünkü benim api_key i de cikti olarak veriyor

In [ ]:
# ! pip install faiss-cpu

In [23]:
docsearch = FAISS.from_texts(texts, embeddings)
docsearch

In [24]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [25]:
chain = load_qa_chain(OpenAI(), chain_type='stuff')

In [27]:
query = 'who are the authors of this article?'
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Yuvanesh Anand, Zach Nussbaum, Brandon Duderstadt, Benjamin Schmidt, and Andriy Mulyar.'

In [29]:
query_2 = 'How was the model trained?'
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query_2)

' The model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.'

In [30]:
query_3 = 'How is different from other models?'
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query_3)

' GPT4All is different from other models because it is trained on a massive curated corpus of assistant interactions, including word problems, story descriptions, multi-turn dialogue, and code. Additionally, the collected data, data curation procedure, training code, and final model weights are openly released to promote open research and reproducibility. Quantized 4-bit versions of the model are also released, allowing virtually anyone to run the model on CPU.'

In [31]:
query_4 = 'What is the BARD?'
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query_4)

# PDF lerde olmayan bilgileri bilemiyor

' There is no BARD mentioned in this context, so the answer is not known.'